Ön hazırlık (kütüphane kurulumu)

In [4]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.7 MB/s eta 0:00:00


TROTTER- SUZUKI YAKLAŞIMI

Cirq ile Trotter–Suzuki Simülasyonu

In [5]:
import cirq
import numpy as np

# 1 qubit
q = cirq.GridQubit(0,0)

# Hamiltonyen terimleri: H = X + Z
H_terms = [cirq.X(q), cirq.Z(q)]
t_total = 1.0
r = 5  # Trotter adımı

# İlk mertebe Trotter ayrışımı
circuit = cirq.Circuit()
dt = t_total / r
for _ in range(r):
    for H in H_terms:
        circuit.append(cirq.rx(2*dt)(q) if isinstance(H, cirq.XPowGate) else cirq.rz(2*dt)(q))

print("Cirq Trotter Devresi:\n", circuit)


Cirq Trotter Devresi:
 (0, 0): ───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───Rz(0.127π)───


Qiskit ile Trotter–Suzuki Simülasyonu

In [6]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np
from scipy.linalg import expm

# 1 qubit Hamiltonyen: H = X + Z
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
H_terms = [X,Z]

t_total = 1.0
r = 5
dt = t_total / r

qc = QuantumCircuit(1)
for _ in range(r):
    for H in H_terms:
        U = expm(-1j * H * dt)
        qc.unitary(Operator(U), [0])

print(qc.draw())


   ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
q: ┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├»
   └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
«   ┌─────────┐┌─────────┐┌─────────┐┌─────────┐
«q: ┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├
«   └─────────┘└─────────┘└─────────┘└─────────┘


QuTiP ile Trotter–Suzuki Simülasyonu

In [7]:
import qutip as qt
import numpy as np

# 1 qubit Hamiltonyen
X = qt.sigmax()
Z = qt.sigmaz()
H_terms = [X,Z]

t_total = 1.0
r = 5
dt = t_total / r

psi0 = qt.basis(2,0)  # Başlangıç durumu |0>
psi = psi0

for _ in range(r):
    for H in H_terms:
        U = (-1j * H * dt).expm()
        psi = U * psi

print("Final state:", psi)


Final state: Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.160631  -0.69086362j]
 [0.14004499-0.69086362j]]


VARIATIONAL QUANTUM SIMULATION(VQS)

Cirq ile VQS Örneği (Basit 1-Qubit

In [8]:
import cirq
import numpy as np

# 1 qubit
q = cirq.GridQubit(0,0)

# Parametrik ansatz: RX ve RZ kapıları
theta = np.pi/4
phi = np.pi/3
circuit = cirq.Circuit(
    cirq.rx(theta)(q),
    cirq.rz(phi)(q)
)

# Ölçüm
circuit.append(cirq.measure(q, key='m'))
print("Cirq VQS Devresi:\n", circuit)


Cirq VQS Devresi:
 (0, 0): ───Rx(0.25π)───Rz(0.333π)───M('m')───


Qiskit ile VQS Örneği

In [9]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

# Parametreler
theta = Parameter('θ')
phi = Parameter('φ')

qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.rz(phi, 0)
qc.measure_all()
print(qc.draw())


        ┌───────┐┌───────┐ ░ ┌─┐
     q: ┤ Rx(θ) ├┤ Rz(φ) ├─░─┤M├
        └───────┘└───────┘ ░ └╥┘
meas: 1/══════════════════════╩═
                              0 


QuTiP ile VQS Örneği (Hamiltonyen Tabanlı)

In [10]:
import qutip as qt
import numpy as np
from scipy.integrate import solve_ivp

# 1 qubit Hamiltonyen: H = X + Z
H = qt.sigmax() + qt.sigmaz()
psi0 = qt.basis(2,0)

# Ansatz: |ψ(θ,φ)> = Rx(θ) Rz(φ) |0>
def ansatz(theta, phi):
    return qt.rx(theta) * qt.rz(phi) * psi0

# McLachlan prensibi için örnek diferansiyel denklem
def dtheta_dt(t, y):
    theta, phi = y
    psi = ansatz(theta, phi)
    # Basit örnek: θ ve φ’nin değişimi, H etkisi ile orantılı
    dtheta = np.imag((psi.dag() * H * psi)[0,0])
    dphi = np.real((psi.dag() * H * psi)[0,0])
    return [dtheta, dphi]

sol = solve_ivp(dtheta_dt, [0,1], [0.1,0.1], t_eval=np.linspace(0,1,10))
print("Parametre evrimi:\n", sol.y)


AttributeError: module 'qutip' has no attribute 'rx'

QUANTUM METROPOLİS ALGORİTMALASI

Cirq ile Basit Quantum Metropolis Yaklaşımı

In [11]:
import cirq
import numpy as np

# 1 qubit Hamiltonyen: H = Z
q = cirq.GridQubit(0,0)
theta = np.pi/4  # başlangıç fazı

circuit = cirq.Circuit(
    cirq.rx(theta)(q)
)

# Ölçüm ile basit enerji örneklemesi
circuit.append(cirq.measure(q, key='m'))
print(circuit)


(0, 0): ───Rx(0.25π)───M('m')───


Qiskit ile Quantum Metropolis

In [12]:
from qiskit import QuantumCircuit, Aer, transpile, execute
from qiskit.circuit import Parameter
import numpy as np

theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.rx(theta, 0)
qc.measure_all()

# Simülasyon
backend = Aer.get_backend('qasm_simulator')
job = execute(qc.bind_parameters({theta: np.pi/4}), backend, shots=100)
counts = job.result().get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Quantum Metropolis (Hamiltonyen Tabanlı)

In [13]:
import qutip as qt
import numpy as np

# Hamiltonyen
H = qt.sigmaz()  # 1-qubit örnek
beta = 1.0  # ters sıcaklık

# Gibbs durumu
rho_beta = qt.ket2dm(qt.basis(2,0)) * np.exp(-beta*H.full())
rho_beta = rho_beta / np.trace(rho_beta)
print("Gibbs Durumu:\n", rho_beta)


TypeError: operand 'Qobj' does not support ufuncs (__array_ufunc__=None)

QUANTUM ANNEALING

Cirq ile Basit Quantum Annealing Simülasyonu

In [14]:
import cirq
import numpy as np

# 1 qubit örneği
q = cirq.GridQubit(0,0)
circuit = cirq.Circuit()

# Başlangıç: H_B = X
circuit.append(cirq.H(q))

# Basit adyabatik interpolasyon (s(t) = t/T)
T = 10
for t in range(T):
    s = t/T
    theta = np.pi * s
    # H_P = Z
    circuit.append(cirq.rz(2*theta)(q))

circuit.append(cirq.measure(q, key='m'))
print(circuit)


(0, 0): ───H───Rz(0)───Rz(0.2π)───Rz(0.4π)───Rz(0.6π)───Rz(0.8π)───Rz(π)───Rz(1.2π)───Rz(1.4π)───Rz(1.6π)───Rz(1.8π)───M('m')───


Qiskit ile Quantum Annealing Yaklaşımı

In [15]:
from qiskit import QuantumCircuit, Aer, execute
import numpy as np

qc = QuantumCircuit(1)
T = 10

# Başlangıç durumunu hazırlama (H_B)
qc.h(0)

for t in range(T):
    s = t / T
    # H_P = Z
    qc.rz(2*np.pi*s, 0)

qc.measure_all()

backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=100).result()
counts = result.get_counts()
print(counts)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP ile Hamiltonyen Tabanlı QA Simülasyonu

In [16]:
import qutip as qt
import numpy as np

# Pauli matrisleri
sx = qt.sigmax()
sz = qt.sigmaz()

# Başlangıç ve problem Hamiltonyeni
H_B = sx
H_P = sz

# Zaman adımları
T = 10
psi0 = qt.basis(2,0)  # başlangıç durumu |0>

psi = psi0
for t in range(T):
    s = t / T
    H_t = (1-s)*H_B + s*H_P
    U = (-1j * H_t).expm()  # kısa zaman adımı için unitary
    psi = U * psi

print("Son durum:\n", psi)


Son durum:
 Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[-0.19379928-0.63507043j]
 [ 0.24061271-0.70797804j]]
